<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark
!pip install pyspark
!pip install findspark

In [3]:
# Start session
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext()

spark = SparkSession \
    .builder \
    .appName("Final Project in SparkML Model").getOrCreate()

24/05/30 02:07:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/30 02:07:17 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Download The search term dataset from the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2024-05-30 02:09:45--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.004s  

2024-05-30 02:09:45 (62.2 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [5]:
# Load the csv into a spark dataframe
searchterms_data = spark.read.csv("searchterms.csv")

In [6]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)
print(f'Total Number of Rows in SearchTerms Dataset : {searchterms_data.count()}')
print(f'Total Number of columns in SearchTerms Dataset : {len(searchterms_data.columns)}')

[Stage 1:>                                                          (0 + 1) / 1]

Total Number of Rows in SearchTerms Dataset : 10001
Total Number of columns in SearchTerms Dataset : 4


In [7]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)
searchterms_data.show(5)

+---+-----+----+--------------+
|_c0|  _c1| _c2|           _c3|
+---+-----+----+--------------+
|day|month|year|    searchterm|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
+---+-----+----+--------------+
only showing top 5 rows



In [11]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)
print(searchterms_data.schema['_c3'].dataType)

StringType


In [12]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)
searchterms_data.createOrReplaceTempView('search_terms')
spark.sql('SELECT COUNT(*) FROM search_terms WHERE _c3 = "gaming laptop"').show()

+--------+
|count(1)|
+--------+
|     499|
+--------+



In [14]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)
spark.sql('SELECT _c3, COUNT(_c3) FROM search_terms GROUP BY _c3 ORDER BY COUNT(_c3) DESC').show(5)

[Stage 9:===============================================>       (172 + 9) / 200]

+-------------+----------+
|          _c3|count(_c3)|
+-------------+----------+
|mobile 6 inch|      2312|
|    mobile 5g|      2301|
|mobile latest|      1327|
|       laptop|       935|
|  tablet wifi|       896|
+-------------+----------+
only showing top 5 rows



In [15]:
# The pretrained sales forecasting model is available at  the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!tar -xvzf model.tar.gz

--2024-05-30 02:29:47--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2024-05-30 02:29:47 (10.5 MB/s) - ‘model.tar.gz’ saved [1490/1490]

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction

In [18]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)
sales_prediction_model = spark.read.parquet('sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet')
sales_prediction_model.printSchema()

root
 |-- intercept: double (nullable = true)
 |-- coefficients: vector (nullable = true)
 |-- scale: double (nullable = true)



In [19]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel

model = LinearRegressionModel.load('sales_prediction.model')

def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','sales')
    predictions = model.transform(__)
    predictions.select('prediction').show()
    
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



24/05/30 02:42:38 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/05/30 02:42:38 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
